In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string
from string import digits

In [2]:
df_train = pd.read_csv("../data/train.csv")
# 3 столбца - id, text, author
df_train.head(n=3)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP


In [3]:
remove_digits = str.maketrans('', '', digits)
def tokenize_stem(file_text):
    #firstly let's apply nltk tokenization
    file_text = file_text.translate(remove_digits)
    try:
        tokens = nltk.word_tokenize(file_text)
    except:
        nltk.download('punkt')
        tokens = nltk.word_tokenize(file_text)
        

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #deleting stop_words
    try:
        stop_words = stopwords.words('english')
    except LookupError:
        nltk.download('stopwords')
        stop_words = stopwords.words('english')
    tokens = [i for i in tokens if ( i not in stop_words )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return tokens

In [4]:
df_train['cleaned_text'] = df_train.text.apply(tokenize_stem)
df_train['cleaned_text_string'] = df_train.cleaned_text.apply(' '.join)
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...


In [5]:
df_train['length']=df_train['cleaned_text_string'].apply(len)
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116


In [6]:
df_hpl=df_train[df_train['author']=='HPL']
df_hpl.describe()

,length
count,5635.000000
mean,93.395386
std,51.075096
min,7.000000
25%,58.000000
50%,85.000000
75%,118.000000
max,561.000000


In [7]:
df_eap=df_train[df_train['author']=='EAP']
df_eap.describe()

,length
count,7900.000000
mean,81.543165
std,60.100183
min,5.000000
25%,40.000000
50%,66.000000
75%,106.000000
max,925.000000


In [8]:
df_mws=df_train[df_train['author']=='MWS']
df_mws.describe()

,length
count,6044.000000
mean,86.124586
std,71.976281
min,4.000000
25%,48.000000
50%,74.000000
75%,108.000000
max,2715.000000


In [41]:
# как мы будем эту штуку правильнее делать (возможно это жуткий костыль), я хз
# сначала создаем словарь где ключ - уникальное слово, а значение - его порядковый номер
# затем создаем разреженную матрицу, которую заполняем в зависимости от порядковых номеров 
word_dict = {}



In [42]:
#делаю сет со всеми словами
# и сразу заготовку под шапку(потом увидишь зачем)
counter = 0
head = []

for wordlist in df_train['cleaned_text']:
    for word in wordlist:
        if word not in word_dict:
            head.append(word)
            word_dict[word] = counter
            counter += 1


In [21]:
# видоизменять колонки в pandas руками по одному значению в строке или столбце - очень плохая идея
# колонка это numpy.ndarray, а значит при каждой итерации она будет пересоздаваться
# что угробит производительность
# делаем значит так. считаем где сколько и где встречались отдельные слова, затем создаем строку за строкой для 
# каждого предложения

list_of_lists = []

for wordlist in df_train['cleaned_text']:
    row = [0 for i in range(len(word_dict))]
    for word in wordlist:
        row[word_dict[word]] += 1
    list_of_lists.append(row)



In [29]:
print(len(list_of_lists))

19579


In [31]:
# ... и для того чтобы посмотреть встречаемость того или иного слова по авторам добавим такую колонку

count_frame = pd.DataFrame(list_of_lists)
count_frame['author'] = df_train['author']



In [44]:
# ... и теперь нормальную шапку делаем

count_frame.columns = head + ['author']


In [14]:
#Считаем повторения
for i in df_mws.index:
    for word in df_mws['cleaned_text'][i]:
        df_word.loc[df_word["word"]==word,"mws"]+=1
        
for i in df_eap.index:
    for word in df_eap['cleaned_text'][i]:
        df_word.loc[df_word["word"]==word,"eap"]+=1
        
for i in df_hpl.index:
    for word in df_hpl['cleaned_text'][i]:
        df_word.loc[df_word["word"]==word,"hpl"]+=1

KeyboardInterrupt: 

In [ ]:
print(df_word.loc[df_word["word"]=="sai", "mws"])